In [1]:
import pandas as pd
import datetime

In [2]:
df = pd.read_csv('elec_data.csv', parse_dates=["date"])
df.head()

,date,time,generation,temp,precipitation,wind_speed,wind_direction,humidity,vapor_pressure,local_pressure,sunshine,insolation,snow
0,2018-01-01,1,0.0,2.2,NaN,1.4,50.0,55.0,3.9,1017.2,NaN,NaN,NaN
1,2018-01-01,2,0.0,1.8,NaN,2.1,50.0,55.0,3.8,1017.1,NaN,NaN,NaN
2,2018-01-01,3,0.0,1.9,NaN,0.8,20.0,58.0,4.1,1017.4,NaN,NaN,NaN
3,2018-01-01,4,0.0,1.2,NaN,1.7,20.0,61.0,4.1,1017.8,NaN,NaN,NaN
4,2018-01-01,5,0.0,-0.1,NaN,1.6,50.0,63.0,3.8,1017.9,NaN,NaN,NaN


In [4]:
df['temp'].fillna(method='pad')
df[['temp', 'wind_speed', 'wind_direction', 'vapor_pressure', 'local_pressure', 'humidity']] = df[['temp', 'wind_speed', 'wind_direction', 'vapor_pressure', 'local_pressure', 'humidity']].fillna(method='pad')
df.fillna(0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            35064 non-null  datetime64[ns]
 1   time            35064 non-null  int64         
 2   generation      35064 non-null  float64       
 3   temp            35064 non-null  float64       
 4   precipitation   35064 non-null  float64       
 5   wind_speed      35064 non-null  float64       
 6   wind_direction  35064 non-null  float64       
 7   humidity        35064 non-null  float64       
 8   vapor_pressure  35064 non-null  float64       
 9   local_pressure  35064 non-null  float64       
 10  sunshine        35064 non-null  float64       
 11  insolation      35064 non-null  float64       
 12  snow            35064 non-null  float64       
dtypes: datetime64[ns](1), float64(11), int64(1)
memory usage: 3.5 MB


In [5]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            35064 non-null  datetime64[ns]
 1   time            35064 non-null  int64         
 2   generation      35064 non-null  float64       
 3   temp            35064 non-null  float64       
 4   precipitation   35064 non-null  float64       
 5   wind_speed      35064 non-null  float64       
 6   wind_direction  35064 non-null  float64       
 7   humidity        35064 non-null  float64       
 8   vapor_pressure  35064 non-null  float64       
 9   local_pressure  35064 non-null  float64       
 10  sunshine        35064 non-null  float64       
 11  insolation      35064 non-null  float64       
 12  snow            35064 non-null  float64       
 13  year            35064 non-null  int64         
 14  month           35064 non-null  int64         
dtypes:

In [7]:
df_train = df.loc[:17519]
df_valid = df.loc[17520:26303]
df_test = df.loc[26304:]

X_train = df_train.drop(['date', 'generation'], axis=1)
y_train = df_train['generation']

X_valid = df_valid.drop(['date', 'generation'], axis=1)
y_valid = df_valid['generation']

X_test = df_test.drop(['date', 'generation'], axis=1)
y_test = df_test['generation']

In [8]:
X_train

,time,temp,precipitation,wind_speed,wind_direction,humidity,vapor_pressure,local_pressure,sunshine,insolation,snow,year,month
0,1,2.2,0.0,1.4,50.0,55.0,3.9,1017.2,0.0,0.0,0.0,2018,1
1,2,1.8,0.0,2.1,50.0,55.0,3.8,1017.1,0.0,0.0,0.0,2018,1
2,3,1.9,0.0,0.8,20.0,58.0,4.1,1017.4,0.0,0.0,0.0,2018,1
3,4,1.2,0.0,1.7,20.0,61.0,4.1,1017.8,0.0,0.0,0.0,2018,1
4,5,-0.1,0.0,1.6,50.0,63.0,3.8,1017.9,0.0,0.0,0.0,2018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,20,-3.3,0.0,2.1,20.0,50.0,2.4,1025.6,0.0,0.0,0.0,2019,12
17516,21,-3.4,0.0,2.1,20.0,53.0,2.5,1025.7,0.0,0.0,0.0,2019,12
17517,22,-4.0,0.0,2.2,20.0,53.0,2.4,1025.5,0.0,0.0,0.0,2019,12
17518,23,-4.5,0.0,3.0,50.0,53.0,2.3,1025.4,0.0,0.0,0.0,2019,12


In [48]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor()

In [69]:
df_error = pd.DataFrame({'error':y_valid - model.predict(X_valid),
                         'time':X_valid['time'],
                        'sunshine':X_valid['sunshine'],
                        'snow':X_valid['snow']})

# df_error = df_error[(df_error['time'] >= 8) & (df_error['time'] <= 20)]
df_error

X_error = df_error.drop(['error'], axis=1)
y_error = df_error['error']

error_pred_model = RandomForestRegressor()
error_pred_model.fit(X_error, y_error)
# df_error.tail(50)

RandomForestRegressor()

In [70]:
df_pred = pd.DataFrame({'time':X_test['time'],
                        'generation':y_test,
                        'pred':model.predict(X_test),
                        'snow':X_test['snow']
                       })

df_pred['residual'] = df_pred['generation'] - df_pred['pred']

# df_error_test = pd.DataFrame({'error':y_valid - model.predict(X_valid),
#                          'time':X_valid['time'],
#                         'sunshine':X_valid['sunshine'],
#                         'snow':X_valid['snow']})

X_error_test = pd.DataFrame({'time':X_test['time'],
                            'sunshine':X_test['sunshine'],
                            'snow':X_test['snow']})

df_pred['res_pred'] = error_pred_model.predict(X_error_test)

In [71]:
df_pred

,time,generation,pred,snow,residual,res_pred
26304,1,0.00,12.64560,11.6,-12.64560,-30.732771
26305,2,0.00,7.35360,11.5,-7.35360,-20.721026
26306,3,0.00,2.58960,11.4,-2.58960,-1.429687
26307,4,0.00,0.00000,10.9,0.00000,-0.173119
26308,5,0.00,0.00000,10.6,0.00000,-0.575886
...,...,...,...,...,...,...
35059,20,1921.90,1106.25927,0.0,815.64073,220.273609
35060,21,769.34,896.13760,0.0,-126.79760,544.605528
35061,22,300.88,461.83460,0.0,-160.95460,306.401891
35062,23,76.00,223.58720,0.0,-147.58720,131.685424


In [81]:
df_pred[(df_pred['snow']>=4.0) & (df_pred['time'] >=8) & (df_pred['time'] <= 20)].head(50)

,time,generation,pred,snow,residual,res_pred
26311,8,0.000,45.24068,10.6,-45.24068,-101.323346
26312,9,117.120,1764.42942,10.6,-1647.30942,-1047.834978
26313,10,671.142,2396.85729,10.6,-1725.71529,-1468.725289
26314,11,1247.013,3919.18565,10.5,-2672.17265,-3883.572559
26315,12,2642.177,17528.61997,8.5,-14886.44297,-14154.815517
26316,13,4132.673,18177.64370,6.9,-14044.97070,-13132.976650
26317,14,5331.163,17873.69253,5.6,-12542.52953,-11066.905652
26318,15,3925.278,14447.35489,4.4,-10522.07689,-5491.033548
26319,16,2660.625,7749.90884,4.0,-5089.28384,-636.732668
26320,17,904.519,2985.25560,4.0,-2080.73660,868.436494


In [73]:
df_error_time = df_error[(df_error['time'] >= 8) & (df_error['time'] <= 20)]
df_error_time

X_error_time = df_error_time.drop(['error'], axis=1)
y_error_time = df_error_time['error']

X_error_time

error_pred_model_time = RandomForestRegressor()
error_pred_model_time.fit(X_error_time, y_error_time)
# # df_error.tail(50)

RandomForestRegressor()

In [75]:
df_pred_time = df_pred[(df_pred['time'] >= 8) & (df_pred['time'] <= 20)]
df_pred_time

X_error_test_time = X_error_test[(X_error_test['time'] >= 8) & (X_error_test['time'] <= 20)]
X_error_test_time
df_pred_time['res_pred'] = error_pred_model_time.predict(X_error_test_time)

C:\Users\hayun\AppData\Local\Temp\ipykernel_17208\1277471467.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred_time['res_pred'] = error_pred_model_time.predict(X_error_test_time)


In [80]:
df_pred_time[df_pred_time['snow']>=4.0].head(50)

,time,generation,pred,snow,residual,res_pred
26311,8,0.000,45.24068,10.6,-45.24068,-196.010011
26312,9,117.120,1764.42942,10.6,-1647.30942,-983.164533
26313,10,671.142,2396.85729,10.6,-1725.71529,-1743.788721
26314,11,1247.013,3919.18565,10.5,-2672.17265,-5238.294435
26315,12,2642.177,17528.61997,8.5,-14886.44297,-7300.916643
26316,13,4132.673,18177.64370,6.9,-14044.97070,-5348.550425
26317,14,5331.163,17873.69253,5.6,-12542.52953,-5456.042717
26318,15,3925.278,14447.35489,4.4,-10522.07689,-795.518999
26319,16,2660.625,7749.90884,4.0,-5089.28384,1133.817127
26320,17,904.519,2985.25560,4.0,-2080.73660,1096.951816
